In [1]:
import pandas as pd
from collections import Counter
import csv
import requests
import random
import pickle
import matplotlib.pyplot as plt
import numpy as np

In [2]:
og_w_b71 = '../pipeline_methods/Orthogroups.tsv'
og_w_b71_df = pd.read_csv(og_w_b71, dtype='string', sep='\t', index_col = 0)

In [3]:
## to get the og that a gene belongs to for orthogrouping without guy11
df_lol = og_w_b71_df.values.tolist()
og_dict_w_b71 = {}

for i, row in enumerate(df_lol):
    og = og_w_b71_df.index[i]
    for cell in row:
        if not pd.isnull(cell):
            for protein in cell.split(', '):
                og_dict_w_b71[protein] = og

In [4]:
## to get all genes associated with one OG for orthogrouping without guy11
genes_per_og_w_b71 = {}

for gene in og_dict_w_b71:
    og = og_dict_w_b71[gene]
    if og not in genes_per_og_w_b71:
        genes_per_og_w_b71[og] = []
    genes_per_og_w_b71[og].append(gene)

In [5]:
input_files = [
    'B71_b71_assembly_complete_medium_expression.txt',
    'B71_b71_assembly_complete_medium_in_planta_expression.txt'
]

output_files =[
    'b71_assembly_complete_medium_expression_per_og.txt',
    'b71_assembly_complete_medium_in_planta_expression_per_og.txt'
]

In [6]:
i=1

input_file = input_files[i]
print(input_file)
output_file = output_files[i]
signal_per_gene_dict = {}
with open(input_file, newline = '') as file:
    file_reader = csv.reader(file, delimiter = '\t')
    for row in file_reader:
        if "ID=" in row[0]:
            gene = row[0].split(';')[0][3:]
        else:
            gene = row[0][:-2]
        signal_per_gene_dict[gene] = float(row[1])

B71_b71_assembly_complete_medium_in_planta_expression.txt


In [7]:
og_signal_w_b71 = {}
for og in genes_per_og_w_b71:
    og_signal_w_b71[og] = []
    for gene in genes_per_og_w_b71[og]:
        if "GCA004785725.2" in gene:
            og_signal_w_b71[og].append(signal_per_gene_dict[gene])

In [8]:
og_signal_w_b71

{'OG0000000': [23.2508,
  43.3103,
  0.0,
  70.7188,
  3.26344,
  19.6297,
  16.3581,
  0.0,
  71.4909,
  26.5401,
  0.0,
  72.4109,
  5.35397,
  3.54617,
  13.0653],
 'OG0000001': [68.7992, 0.330731, 1.0268, 5.42273, 21.549, 0.0],
 'OG0000002': [13.1266, 38.6113, 0.423693, 0.139814],
 'OG0000003': [0.0, 0.0, 0.0, 11.858],
 'OG0000004': [339.543, 105.323, 0.0, 185.499, 113.533],
 'OG0000005': [77.7483, 0.0, 54.8754, 0.0, 0.0, 0.0],
 'OG0000006': [3.07815, 0.596221, 0.0],
 'OG0000007': [4.69507, 1.28352, 0.0, 0.038334],
 'OG0000008': [0.639187, 0.0, 0.0, 0.0],
 'OG0000009': [0.168147, 0.0861033],
 'OG0000010': [28.915, 25.0682, 42.1822, 0.0, 0.614853, 0.0],
 'OG0000011': [10.0618, 1.4556, 10.0172, 13.039, 0.0831857],
 'OG0000012': [181.615, 37.9733, 0.0, 0.0, 8.55492, 422.964],
 'OG0000013': [0.0, 0.24972],
 'OG0000014': [40.3139, 0.0, 0.0],
 'OG0000015': [0.680806, 0.0, 10.9352],
 'OG0000016': [16.9633, 0.0],
 'OG0000017': [0.489979],
 'OG0000018': [0.0, 7.15874],
 'OG0000019': [0.4581

In [9]:
og_signal_w_b71_averaged = {}
for og in og_signal_w_b71:
    lst = og_signal_w_b71[og]
    if len(lst) > 1:
        og_signal_w_b71_averaged[og] = sum(lst) / len(lst)
    elif len(lst) == 1:
        og_signal_w_b71_averaged[og] = lst[0]
    elif len(lst) == 0:
        pass
    else:
        print('wtf')

In [10]:
og_signal_w_b71_averaged

{'OG0000000': 24.595898666666663,
 'OG0000001': 16.18807683333333,
 'OG0000002': 13.07535175,
 'OG0000003': 2.9645,
 'OG0000004': 148.77960000000002,
 'OG0000005': 22.103949999999998,
 'OG0000006': 1.2247903333333332,
 'OG0000007': 1.504231,
 'OG0000008': 0.15979675,
 'OG0000009': 0.12712515,
 'OG0000010': 16.130042166666666,
 'OG0000011': 6.93135714,
 'OG0000012': 108.51787,
 'OG0000013': 0.12486,
 'OG0000014': 13.437966666666666,
 'OG0000015': 3.872002,
 'OG0000016': 8.48165,
 'OG0000017': 0.489979,
 'OG0000018': 3.57937,
 'OG0000019': 0.2563044,
 'OG0000020': 1.4546610000000002,
 'OG0000021': 26.548218000000002,
 'OG0000022': 17.53194,
 'OG0000023': 16.8898,
 'OG0000024': 7.416398766666666,
 'OG0000025': 15.342980650000001,
 'OG0000026': 0.123702,
 'OG0000027': 0.7054314666666667,
 'OG0000028': 5.272403333333333,
 'OG0000029': 31.278594,
 'OG0000030': 18.972875,
 'OG0000031': 65.89116666666666,
 'OG0000032': 1.17548,
 'OG0000033': 8.927029000000001,
 'OG0000034': 0.307067,
 'OG00000

In [11]:
median_value = np.median(list(og_signal_w_b71_averaged.values()))
imputed_values = []
for og in genes_per_og_w_b71:
    if og not in og_signal_w_b71_averaged:
        imputed_values.append(og)
        og_signal_w_b71_averaged[og] = median_value
print(len(imputed_values))
with open(output_file, 'w', newline = '') as output_csv:
    w = csv.writer(output_csv, delimiter = '\t')
    for key in og_signal_w_b71_averaged:
        w.writerow([key, og_signal_w_b71_averaged[key]])

4289


In [13]:
for i in range(len(input_files)):
    input_file = input_files[i]
    print(input_file)
    output_file = output_files[i]
    signal_per_gene_dict = {}
    with open(input_file, newline = '') as file:
        file_reader = csv.reader(file, delimiter = '\t')
        for row in file_reader:
            if "ID=" in row[0]:
                gene = row[0].split(';')[0][3:]
            else:
                gene = row[0][:-2]
            signal_per_gene_dict[gene] = float(row[1])
    og_signal_w_b71 = {}
    for og in genes_per_og_w_b71:
        og_signal_w_b71[og] = []
        for gene in genes_per_og_w_b71[og]:
            if "GCA004785725.2" in gene:
                og_signal_w_b71[og].append(signal_per_gene_dict[gene])
    og_signal_w_b71_averaged = {}
    for og in og_signal_w_b71:
        lst = og_signal_w_b71[og]
        if len(lst) > 1:
            og_signal_w_b71_averaged[og] = sum(lst) / len(lst)
        elif len(lst) == 1:
            og_signal_w_b71_averaged[og] = lst[0]
        elif len(lst) == 0:
            pass
        else:
            print('wtf')
    median_value = np.median(list(og_signal_w_b71_averaged.values()))
    imputed_values = []
    for og in genes_per_og_w_b71:
        if og not in og_signal_w_b71_averaged:
            imputed_values.append(og)
            og_signal_w_b71_averaged[og] = median_value
    print(len(imputed_values))
    with open(output_file, 'w', newline = '') as output_csv:
        w = csv.writer(output_csv, delimiter = '\t')
        for key in og_signal_w_b71_averaged:
            w.writerow([key, og_signal_w_b71_averaged[key]])

B71_b71_assembly_complete_medium_expression.txt
4289
B71_b71_assembly_complete_medium_in_planta_expression.txt
4289
